In [1]:
import numpy as np
import torch as th
from torch import autograd

from showerSim import invMass_ginkgo

Cannot import dbm.gnu: No module named '_gdbm'


/scratch/mdd424/CGinkgo/lib/python3.7/site-packages/pyprob/util.py:332: UserWarning: Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.
  warnings.warn('Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.')


In [2]:
jetM = 80. # parent mass -> W
jetdir = th.tensor([1.,1.,1.], dtype=th.double) # direction
jetP = 400. # magnitude
jetvec = jetP * jetdir / th.norm(jetdir)

jet4vec = th.cat((th.tensor([np.sqrt(jetP**2 + jetM**2)], dtype=th.double), jetvec))
jet4vec.requires_grad = True

tensor([407.9216, 230.9401, 230.9401, 230.9401], dtype=torch.float64,
       requires_grad=True)

In [3]:
# Condition on the number of leaves
def num_leaves_cut(self, jet):
    return len(jet["leaves"]) >= 27

In [4]:
simulator = invMass_ginkgo.SimulatorModel(#rate=[3., 1.5], # exponential dsitribution rate
                                     jet_p=jet4vec,  # parent particle 4-vector
                                     pt_cut=10.,  # minimum pT for resulting jet
                                     Delta_0=th.tensor(jetM**2, requires_grad=True),  # parent particle mass squared -> needs tensor
                                     M_hard=jetM,  # parent particle mass
                                     minLeaves=30,  # minimum number of jet constituents
                                     maxLeaves=40,  # maximum " "
                                     bool_func=num_leaves_cut,
                                     suppress_output=True)

## Latent Variable Naming Scheme

Direction of decay in CM frame:
- "phiCM" + "..."
- "thetaCM_U" + "..."

Invariant Mass of decay products:
- "L_decay" + "..."
- "R_decay" + "..."

In [5]:
def is_latent_variable(var):
    latent_var_names = ["phiCM", "thetaCM_U", "L_decay", "R_decay"]
    arr = np.array([var.name.startswith(x) for x in latent_var_names])
    return arr.any()

In [56]:
rate = th.tensor([3, 1.5], dtype=th.double, requires_grad=True)

trace = simulator.get_trace(inputs=rate)
trace.log_prob

tensor(-32.8037, grad_fn=<AddBackward0>)

In [53]:
trace.log_prob.retain_grad()
trace.log_prob.backward(retain_graph=True)

In [54]:
rate.grad

tensor([-0.0081,  3.0347], dtype=torch.float64)

## Do a forward pass that also returns the joint score

In [5]:
rate = th.tensor([3, 1.5], dtype=th.double, requires_grad=True)

data = simulator.augmented_data(rate)

In [ ]:
data;

## Check to see if the expectation of the joint score is zero

In [5]:
rate = th.tensor([3, 1.5], dtype=th.double, requires_grad=True)

running_score = np.zeros(2)
for i in range(10000):
    data = simulator.augmented_data(rate)
    running_score += data["joint_score"].detach().numpy()
    rate.grad.zero_()
    if ((i+1) % 1000) == 0:
        print(" "*20, end="\r")
        print("Iteration:", i)
    if ((i+1) % 50) == 0:
        print("[{:.4f}, {:.4f}]".format(running_score[0]/(i+1), running_score[1]/(i+1)), end="\r")
running_score / 10000

Iteration: 0        
Iteration: 1000     
Iteration: 2000     
Iteration: 3000     
Iteration: 4000     
Iteration: 5000     
Iteration: 6000     
Iteration: 7000     
Iteration: 8000     
Iteration: 9000     


array([0.33807291, 3.31132955])

# THE CODE BELOW IS IN PROGRESS

## Generate a dataset

In [ ]:
theta_dist = th.distributions.Uniform(0.1, 10)

num_thetas = 5 #100
num_samples_

theta1 = th.tensor([3, 1.5], dtype=th.double, requires_grad=True)  # Reference parameter
num_thetas = 5 #100
num_samples_per_theta = 200 #5000
theta2 = theta_dist.sample((num_thetas, 2))

for i in range(num_thetas):
    theta2_1 = theta_dist.sample()
    theta2_2 = theta_dist.sample()
    theta2 = th.tensor([theta2_1, theta2_2], dtype=th.double, requires_grad=True)
    
    for j in range(num_samples_per_theta):
        

## Train a NN to learn the likelihood ratio

In [10]:
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class DeepRegressor(nn.Module):
    def __init__(self, input_dim, num_layers, width, dropout=0.2):
        super(DeepRegressor, self).__init__()
        self.num_layers = num_layers
        self.width = width
        self.p_drop = dropout
        # Layers
        #  - Linear
        #  - Activation
        #  - Batch Normalization
        #  - Dropout
        self.linear_layers = nn.ModuleList()
        self.activation_layers = nn.ModuleList()
        self.norm_layers = nn.ModuleList()
        # Add the first hidden layer
        self.linear_layers.append(nn.Linear(input_dim, input_dim*width))
        self.activation_layers.append(nn.PReLU())
        self.norm_layers.append(nn.BatchNorm1d(input_dim*width))
        # Add the rest
        for i in range(num_layers-1):
            self.linear_layers.append(nn.Linear(input_dim*width, input_dim*width))
            self.activation_layers.append(nn.PReLU())
            self.norm_layers.append(nn.BatchNorm1d(input_dim*width))
        # Initialize the weights for the linear layers in a special way
        self.init_kaiming(self.linear_layers)
        # Output Layer
        self.output_layer = nn.Linear(input_dim*width, 1)
        # Dropout Layer
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x):    
        for i in range(self.num_layers):
            x = self.dropout(self.norm_layers[i](self.activation_layers[i](self.linear_layers[i](x))))
        return self.output_layer(x)
    
    
    @staticmethod
    def init_kaiming(moduleList):
        for linear_layer in moduleList:
            nn.init.kaiming_normal_(linear_layer.weight, a=0.25)
            nn.init.constant_(linear_layer.bias, 0)

In [21]:
th.distributions.Uniform(0.1, 10).sample()

tensor(7.8556)